In [1]:
#Simple feature map to feed arrays into the classifier. 
import numpy as np
import json


In [2]:
f = open('data/test.json')
data = json.load(f)
f.close()

In [25]:
data[0].keys()

dict_keys(['id', 'sent', 'tags'])

In [3]:
def word_shape_features(word):
    return np.array([word.istitle(), word.islower(), word.isupper(), len(word),
                     word.isdigit(),  word.isalpha(),word.isalnum(), word.isnumeric()])

def get_word_features(word):
    return word_shape_features(word)

def get_sent_features(sent):
    ret = []
    for word in sent:
        ret.append(get_word_features(word))
    return ret

In [ ]:
labels = ["abc","def"]


In [13]:
label2id = {}
for id,label in enumerate(labels):
    label2id[label] = id 

def get_labels(tags):
    ret = []
    prev_tag = ""
    for tag in tags:
        if tag == "O":
            ret.append(2*len(labels))
        elif tag == prev_tag:
            ret.append(label2id[tag]+ len(labels))
        else:
            ret.append(label2id[tag])
        prev_tag = tag
    return ret 


In [18]:
get_labels(["O","abc"])

[4, 0]

In [9]:
for id, d in enumerate(data):
    data[id]["features"] = get_sent_features(d["sent"])
    data[id]["labels"] = get_labels(d["tags"])
    # break
    # print(d["sent"])
    # print(d["tags"])

In [11]:
data[0]

{'id': 0,
 'sent': ['A',
  'handful',
  'of',
  'professors',
  'in',
  'the',
  'UW',
  'Department',
  'of',
  'Chemistry',
  'are',
  'being',
  'recognized',
  'by',
  'the',
  'American',
  'Association',
  'for',
  'the',
  'Advancement',
  'of',
  'Science',
  '-LRB-',
  'AAAS',
  '-RRB-',
  'for',
  'their',
  'efforts',
  'and',
  'contributions',
  'to',
  'the',
  'scientific',
  'community',
  '.'],
 'tags': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  ['/organization/educational_institution', '/organization'],
  ['/education/department', '/organization'],
  ['/education/department', '/organization'],
  ['/education/department', '/organization'],
  'O',
  'O',
  'O',
  'O',
  'O',
  ['/organization'],
  ['/organization'],
  ['/organization'],
  ['/organization'],
  ['/organization'],
  ['/organization'],
  ['/organization'],
  'O',
  ['/organization'],
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'features': [array([1, 0, 1, 1, 0, 1, 1, 0]),


In [ ]:
# Word identity
# word embeddings
# Word suffix (last 2 and 3 characters)
# Word prefix (first 2 and 3 characters)
# Whether the word contains a hyphen
# Whether the word contains a dot
# Whether the word contains a slash